In [30]:
#Gaussian Noise for one needle
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def tumor_coverage_1(nrrd_file, center, radii):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()

    image = reader.GetOutput()

    # Create the ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii[0] / Spacing[0], radii[1] / Spacing[1], radii[2] / Spacing[2])

    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()

    # Align the iceball and tumor images
    sp = sphere1.GetOutput()
    sp.SetOrigin(image.GetOrigin())

    # Logic AND operation between iceball and tumor
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(image)
    logic.SetInput2Data(sp)
    logic.SetOperationToAnd()
    logic.SetOutputTrueValue(1)
    logic.Update()

    # Extract scalar data and calculate sum
    sc = logic.GetOutput().GetPointData().GetScalars()
    a = vtk_to_numpy(sc)
    return numpy.sum(a)

In [73]:
import numpy as np

def perturb_with_gaussian_noise(coordinates, std_x, std_y, std_z):
    
    # Ensure coordinates is a tuple or list with three elements
    if not isinstance(coordinates, (tuple, list)) or len(coordinates) != 3:
        raise ValueError("Coordinates should be a tuple or list with three elements (x, y, z).")

    # Unpack the input coordinates
    x, y, z = coordinates

    # Generate random perturbations from Gaussian distributions
    perturb_x = np.random.normal(loc=0, scale=std_x)
    perturb_y = np.random.normal(loc=0, scale=std_y)
    perturb_z = np.random.normal(loc=0, scale=std_z)

    # Apply perturbations to the coordinates
    x_perturbed = x + perturb_x
    y_perturbed = y + perturb_y
    z_perturbed = z + perturb_z

    # Return perturbed coordinates as a tuple
    return (x_perturbed, y_perturbed, z_perturbed)

In [83]:
coordinates = (123, 120, 20)
std_dev_x = 2
std_dev_y = 2
std_dev_z = 2
perturbed_coordinates = perturb_with_gaussian_noise(coordinates, std_dev_x, std_dev_y, std_dev_z)
print("Original Coordinates:", coordinates)
print("Perturbed Coordinates:", perturbed_coordinates)

Original Coordinates: (123, 120, 20)
Perturbed Coordinates: (123.56717960973953, 120.85831131450874, 24.12500436831423)


In [84]:
# Expected Value --> Simulate 100 times and take average???
# Add a counter every time tumor is not fully covered then find percentage of not covering
x = perturbed_coordinates[0]
y = perturbed_coordinates[1]
z = perturbed_coordinates[2]
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
center = [x, y, z]
radii = [10.0, 10.0, 10.0]
sum_result = tumor_coverage_1(nrrd_file, center, radii)
print(sum_result)

1049


In [11]:
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def iceball_coverage_1(nrrd_file, center, radii):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()

    image = reader.GetOutput()

    # Create the ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii[0] / Spacing[0], radii[1] / Spacing[1], radii[2] / Spacing[2])

    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()

    # Align the iceball and tumor images
    sp = sphere1.GetOutput()
    sp.SetOrigin(image.GetOrigin())

    # Logic AND operation between iceball and tumor
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(image)
    logic.SetInput2Data(sp)
    logic.SetOperationToAnd()
    logic.SetOutputTrueValue(1)
    logic.Update()

    # Extract scalar data and calculate iceball coverage
    image_data = vtk_to_numpy(image.GetPointData().GetScalars())
    logic_data = vtk_to_numpy(logic.GetOutput().GetPointData().GetScalars())
    
    image_coverage = numpy.sum(image_data)  # Total coverage of the original image
    iceball_coverage = numpy.sum(logic_data)  # Coverage of the iceball within the image
    
    return iceball_coverage / image_coverage

In [13]:
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
center = [121, 118, 22]
radii = [10.0, 10.0, 10.0]
coverage_ratio = iceball_coverage_1(nrrd_file, center, radii)
print("Iceball coverage ratio:", coverage_ratio)

Iceball coverage ratio: 0.903370786516854


In [69]:
from vtk.util.numpy_support import vtk_to_numpy
'''
Uses SD to find needle coordinates, finds tumor coverage, if tumor not entirely covered add a counter

Arguments: 
    - nrrd file
    - standard deviation in x, y, z directions
    - iceball center, tuple or array with 3 values
    - iceball radii, tuple or array with 3 values

Return: counter / simulation #
1. Loop through # times
2. Perturbed Coordinates
3. Tumor Ratio
4. If below 1.0
    Add to a counter
else keep on looping

Return percentage of tumor coverage

'''

def perturbed_tumor_percentage(nrrd, simulate, std_x, std_y, std_z, center, radii):
    for i in range(simulate):
        # Read the NRRD file using vtkNrrdReader
        reader = vtk.vtkNrrdReader()
        reader.SetFileName(nrrd_file)
        reader.Update()
    
        image = reader.GetOutput()

        # Create the ellipsoid source
        
        # Perturb Center
        x = center[0]
        y = center[1]
        z = center[2]

        # Generate random perturbations from Gaussian distributions
        perturb_x = np.random.normal(loc=0, scale=std_x)
        perturb_y = np.random.normal(loc=0, scale=std_y)
        perturb_z = np.random.normal(loc=0, scale=std_z)

        # Apply perturbations to the coordinates
        x_perturbed = x + perturb_x
        y_perturbed = y + perturb_y
        z_perturbed = z + perturb_z
        
        center = [x_perturbed, y_perturbed, z_perturbed]

        sphere1 = vtk.vtkImageEllipsoidSource()
        sphere1.SetOutputScalarTypeToShort()
        sphere1.SetCenter(center)
        Spacing = image.GetSpacing()
        sphere1.SetRadius(radii[0] / Spacing[0], radii[1] / Spacing[1], radii[2] / Spacing[2])
    
        # Create the dimensions of the file
        size_image = image.GetDimensions()
        sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
        sphere1.Update()

        # Align the iceball and tumor images
        sp = sphere1.GetOutput()
        sp.SetOrigin(image.GetOrigin())

        # Logic AND operation between iceball and tumor
        logic = vtk.vtkImageLogic()
        logic.SetInput1Data(image)
        logic.SetInput2Data(sp)
        logic.SetOperationToAnd()
        logic.SetOutputTrueValue(1)
        logic.Update()

        # Extract scalar data and calculate iceball coverage
        image_data = vtk_to_numpy(image.GetPointData().GetScalars())
        logic_data = vtk_to_numpy(logic.GetOutput().GetPointData().GetScalars())
    
        image_coverage = numpy.sum(image_data)  # Total coverage of the original image
        iceball_coverage = numpy.sum(logic_data)  # Coverage of the iceball within the image
        
        counter = 0
        if iceball_coverage / image_coverage < 1.0:
            counter = counter + 1
        
    return 1 - counter / simulate

In [70]:
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
center = [121, 118, 22]
radii = [1.0, 1.0, 1.0]
perturbed_percentage = perturbed_tumor_percentage(nrrd_file, 100, 2, 2, 2, center, radii)
print("Iceball coverage ratio:", perturbed_percentage)

Iceball coverage ratio: 0.99


In [1]:
import vtk
import numpy as np
from vtk.util.numpy_support import vtk_to_numpy

def perturbed_tumor_percentage_GPT(nrrd_file, simulate, std_x, std_y, std_z, center, radii):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()
    
    image = reader.GetOutput()

    # Create the ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii[0] / Spacing[0], radii[1] / Spacing[1], radii[2] / Spacing[2])
    
    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()

    # Align the iceball and tumor images
    sp = sphere1.GetOutput()
    sp.SetOrigin(image.GetOrigin())

    counter = 0  # Initialize counter outside the loop

    for i in range(simulate):
        x = center[0]
        y = center[1]
        z = center[2]

        # Generate random perturbations from Gaussian distributions
        perturb_x = np.random.normal(loc=0, scale=std_x)
        perturb_y = np.random.normal(loc=0, scale=std_y)
        perturb_z = np.random.normal(loc=0, scale=std_z)

        # Apply perturbations to the coordinates
        x_perturbed = x + perturb_x
        y_perturbed = y + perturb_y
        z_perturbed = z + perturb_z
        
        center = [x_perturbed, y_perturbed, z_perturbed]

        # Logic AND operation between iceball and tumor
        logic = vtk.vtkImageLogic()
        logic.SetInput1Data(image)
        logic.SetInput2Data(sp)
        logic.SetOperationToAnd()
        logic.SetOutputTrueValue(1)
        logic.Update()

        # Extract scalar data and calculate iceball coverage
        image_data = vtk_to_numpy(image.GetPointData().GetScalars())
        logic_data = vtk_to_numpy(logic.GetOutput().GetPointData().GetScalars())
    
        image_coverage = np.sum(image_data)   # Total coverage of the original image
        iceball_coverage = np.sum(logic_data) # Coverage of the iceball within the image
        
        if (iceball_coverage / image_coverage) < 1.0:
            counter = counter + 1

    return counter / simulate  # Return the percentage of tumor coverage



In [15]:
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
simulate = 1000
std_x = 2.0
std_y = 2.0
std_z = 2.0
center = (121, 118, 22)
radii = (12, 12.0, 12.0)

result = perturbed_tumor_percentage_GPT(nrrd_file, simulate, std_x, std_y, std_z, center, radii)
print(f"Tumor Coverage Percentage: {result * 100:.2f}%")

Tumor Coverage Percentage: 100.00%
